# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
DEPLOYMENT_URL="http://localhost:2024/"
# DEPLOYMENT_URL="http://localhost:8000/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data
 
 
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)
 
        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    if msg_type == "ai":
                        return True
    return False

In [3]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': '8a1797a9-9f11-4d8d-8346-4b2786d4f354', 'created_at': '2025-08-14T04:13:17.580052+00:00', 'updated_at': '2025-08-14T04:13:17.580055+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None}
8a1797a9-9f11-4d8d-8346-4b2786d4f354


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [4]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [ ]:
input = { "messages": "한국 반도체 현황. 모두 한글로 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    list_chunk.append(chunk)
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])
    # elif filter_silde_data(chunk):
    #     print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
    #     print(chunk)


##########  AI 메시지  ##########
한국 반도체 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 요약 및 개요 - 보고서 목적과 범위, 핵심 요약(시장규모·수출 비중·주요 지표) 제시. 조사 방법과 데이터 출처 간단 명시. 
3. 시장 규모 및 수출 동향 - 국내 반도체 시장 규모(매출·생산액), 연도별 성장률, 품목별(메모리·파운드리 등) 매출 비중 및 국가별 수출 비중 분석. 
4. 생산능력·파운드리·미세공정 현황 - 팹과 생산능력(웨이퍼·월량), 파운드리 역량, 공정 노드(나노미터) 분포 및 최신 미세공정 도입 현황과 설비 투자(CAPEX) 동향. 
5. 주요 기업과 기술 동향 - 삼성전자·SK하이닉스 등 주요 기업별 포트폴리오(DRAM, NAND, HBM, 파운드리 등), R&D 투자, 패키징·3D 적층·EUV 등 첨단 기술 동향 분석. 
6. 공급망(소재·장비) 현황과 취약점 - 포토레지스트·화학가스·웨이퍼·EUV 장비 등 핵심 소재·장비의 공급망 구조와 해외 의존도, 병목 요인 및 국내 대체전략 분석. 
7. 정부 정책·지원과 K-반도체 전략 - 정부의 반도체 산업정책, 예산·인센티브, K-반도체 벨트 등 지역·산업 육성 전략, 법·규제적 지원 현황과 계획 정리. 
8. 글로벌 경쟁구도·무역·수출규제 - 대만·미국·중국 등과의 경쟁구도, 글로벌 파운드리 경쟁 현황, 미·중 기술·수출규제와 지정학적 리스크 영향 분석. 
9. 주요 리스크 및 과제 - 인력 부족, 설비 투자 부담, 공급망 취약성, 기술격차, 지정학적·무역 리스크 등 단기·중장기 리스크와 해결 과제 정리. 
10. 전망 및 권고안 - 단기·중장기 산업 전망과 시사점, 정책·기업 차원의 권고사항(투자 우선순위, R&D·인력 양성·공급망 다변화·국제협력 등) 제

In [ ]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [ ]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

In [ ]:
len(state["values"]["slides"])

In [ ]:
state["values"]["slides"]

In [ ]:
for s in state["values"]["slides"]:
    with open(f"{s['idx']}.html", "w", encoding="utf-8") as f:
        f.write(s['html'])